In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os

os.environ['SPOTIPY_CLIENT_ID']='3e8627e438fa4781abf064e0d3bc7b6f'
os.environ['SPOTIPY_CLIENT_SECRET']='0ac3077f50b54e248fc32c64c8fc9e13'

# initialize spotipy
client_credentials_manager = SpotifyClientCredentials()
spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# read in data from playlist_name_cleaned.csv
# df = pd.read_csv('cleaned_playlist_uri.csv')

In [5]:
import fasttext 
from fasttext import train_supervised
from cleantext import clean
import nltk 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import emoji, re

def clean_text(text):
    text = clean(text, no_line_breaks=True, no_urls=True, no_punct=True, no_digits=True, replace_with_punct="", replace_with_digit="")
    text = emoji.replace_emoji(text, replace='')
    text = text.strip()
    text = text.replace('~', '')
    text = text.replace('_', '')
    text = text.replace('$', '')
    text = text.replace('^', '')
    # remove all mathemtical symbols, including =, +, -, *, /, <, >, %
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text_tokens = word_tokenize(text)
    cleaned_tokens = [w for w in text_tokens if not w in stopwords.words('english')]
    text = (" ").join(cleaned_tokens)
    if len(text) < 3:
        return ""
    return text

print(len(df))
# rename playlist_names with cleaned text, remove empty rows
df['playlist_name'] = df['playlist_name'].apply(clean_text)
df = df[df['playlist_name'] != ""]
print(len(df))

# save to a new csv file
df.to_csv('cleaner_playlist_uri.csv', index=False)

# convert df to dictionary
playlist_dict = df.set_index('playlist_name').T.to_dict('list')

for key in list(playlist_dict.keys()):
    tracklist = eval(playlist_dict[key][0])
    playlist_dict[key] = tracklist

# place each key on a new line in a text file
with open('clean_playlist.txt', 'w') as f:
    for key in playlist_dict.keys():
        f.write(key + ' ')
        for track in playlist_dict[key]:
            f.write('__label__' + track + ' ')
        f.write('\n')

53733
53733


/var/folders/v1/ljzyxbgj1xx_syzq56dgnsk00000gn/T/ipykernel_1485/3243427280.py:37: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  playlist_dict = df.set_index('playlist_name').T.to_dict('list')


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [1]:
import random
# randomly split the data into 80% training and 20% testing
with open('clean_playlist.txt', 'r') as f:
    data = f.readlines()
    random.shuffle(data)
    train = data[:int(len(data)*0.8)]
    test = data[int(len(data)*0.8):]
    # write train.txt and test.txt
    with open('playlist.train', 'w') as f:
        for line in train:
            f.write(line)
    with open('playlist.test', 'w') as f:
        for line in test:
            f.write(line)

In [1]:
import fasttext 
from fasttext import train_supervised

# train fasttext model on the playlist names, using track uri as the label
model = fasttext.train_supervised(input='clean_playlist.txt', lr=0.5, epoch=500, wordNgrams=4, bucket=200000, dim=50, loss='hs')

Read 2M words
Number of words:  8818
Number of labels: 558710
Progress: 100.0% words/sec/thread: 1263230 lr:  0.000000 avg.loss: 11.029614 ETA:   0h 0m 0s 37.8% words/sec/thread: 1276024 lr:  0.311233 avg.loss: 13.775496 ETA:   0h 1m36s 52.0% words/sec/thread: 1233817 lr:  0.240097 avg.loss: 12.900927 ETA:   0h 1m17s 86.0% words/sec/thread: 1267403 lr:  0.070012 avg.loss: 11.465023 ETA:   0h 0m21s


In [9]:
df = pd.read_csv('cleaner_playlist_uri.csv')

In [25]:
prediction_words = "sad depressing feels"
# predict the track uri's for the given words
track_predictions = model.predict(prediction_words, k=20)
# select the first 10 unique predicted track uri's
track_predictions = np.unique(track_predictions[0][:10], return_counts=True)

# get the track names for the predicted track uri's
for track in track_predictions[0]:
    print(spotify.track(track[9:])['name'])

# get a new dataframe containing only playlists with any of the words in the playlist name
rslt_df = df.loc[df['playlist_name'].str.contains(prediction_words.replace(' ', ' | ')) == True]
playlist_dict = rslt_df.set_index('playlist_name').T.to_dict('list')

for key in list(playlist_dict.keys()):
    tracklist = eval(playlist_dict[key][0])
    playlist_dict[key] = tracklist

accuracy = []
# for each top 10 predicted track, add 1 to the list if it is in one of the playlists, else add 0
for track in track_predictions[0]:
    for key in playlist_dict.keys():
        if track[9:] in playlist_dict[key]:
            accuracy.append(1)
            print(spotify.track(track[9:])['name'] + " is in " + key)
            break

# calculate the accuracy
print(sum(accuracy)/10)

Give Me Love
Kiss Me
Thinking out Loud
Ho Hey
Slow Dancing in a Burning Room
Hey There Delilah
Skinny Love
Lay Me Down
Photograph
Breathe Me


/var/folders/v1/ljzyxbgj1xx_syzq56dgnsk00000gn/T/ipykernel_6883/3063602081.py:13: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  playlist_dict = rslt_df.set_index('playlist_name').T.to_dict('list')


Give Me Love is in sad music
Kiss Me is in sad days
Thinking out Loud is in sad love songs
Ho Hey is in sad love songs
Slow Dancing in a Burning Room is in sad af
Hey There Delilah is in sad days
Skinny Love is in sad days
Lay Me Down is in sad days
Photograph is in sad days
Breathe Me is in sad songs
1.0


In [47]:
import random
# from the set of all playlist names, get the unique words
all_words = set()
for playlist in df['playlist_name']:
    for word in str(playlist).split():
        all_words.add(word)

# create a random list of 1000 sets of 1-5 words from the set of all words
random_descriptions = []
for i in range(1000):
    description_words = random.sample(all_words, random.randint(1, 5))
    description = ' '.join(description_words)
    random_descriptions.append(description)

overall_accuracy = []
# # predict the track uri's for the given words
for description in random_descriptions:
    track_predictions = model.predict(description, k=20)
    track_predictions = np.unique(track_predictions[0][:10], return_counts=True)
    # get the track names for the predicted track uri's
    for track in track_predictions[0]:
        print(spotify.track(track[9:])['name'])
    # get a new dataframe containing only playlists with any of the words in the playlist name
    rslt_df = df.loc[df['playlist_name'].str.contains(description.replace(' ', ' | ')) == True]
    playlist_dict = rslt_df.set_index('playlist_name').T.to_dict('list')
    for key in list(playlist_dict.keys()):
        tracklist = eval(playlist_dict[key][0])
        playlist_dict[key] = tracklist
    accuracy = []
    # for each top 10 predicted track, add 1 to the list if it is in one of the playlists, else add 0
    print("Input Description: " + description + "\n")
    print("Reccomended Tracks: ")
    for track in track_predictions[0]:
        print(spotify.track(track[9:])['name'])
        for key in playlist_dict.keys():
            if track[9:] in playlist_dict[key]:
                accuracy.append(1)
                break
    # calculate the accuracy
    overall_accuracy.append(sum(accuracy)/10)
    # write to a text file
    with open('fasttext_results.txt', 'a') as f:
        f.write(str(sum(accuracy)/10) + '\n')

print(sum(overall_accuracy)/1000)

/var/folders/v1/ljzyxbgj1xx_syzq56dgnsk00000gn/T/ipykernel_6883/3317257751.py:11: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  description_words = random.sample(all_words, random.randint(1, 5))


Centuries
Seven Nation Army
I Miss You
Demons
Chop Suey!
Last Resort
How to Save a Life
In the End
Radioactive
7 Years
In the Name of Love
That's What I Like
September
Demons
Don't Wanna Know (feat. Kendrick Lamar)
I Want You Back
Mercy
My House
Closer
Shape of You
Hotline Bling
Five Nights at Freddy's: Night 5
Paradise City
Breezeblocks
Hotel California - 2013 Remaster
Five Nights at Freddy's: Night 3
Radioactive
Wake Me Up - Radio Edit
Sail
Sweet Child O' Mine
Titanium (feat. Sia)
Timber (feat. Ke$ha)
Limbo
Lean On (feat. MØ & DJ Snake)
Fireball (feat. John Ryan)
Happy - From "Despicable Me 2"
Counting Stars
Animals - Extended
Bad (feat. Vassy) - Radio Edit
Five Hours
We Will All Be Changed
Holocene
Fast Car
The Girl
Holy River
Argue With Heaven (feat. Summer Ames)
Coattails
Constant Headache
Feels Like We Only Go Backwards
Say Yes
We Will All Be Changed is in history
Holy River is in history
Argue With Heaven (feat. Summer Ames) is in history


/var/folders/v1/ljzyxbgj1xx_syzq56dgnsk00000gn/T/ipykernel_6883/3317257751.py:26: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  playlist_dict = rslt_df.set_index('playlist_name').T.to_dict('list')


The Lord Is a Warrior (feat. Shai Linne)
Charleston
Tom's Diner - Reprise
You Never Fail Us
Lord, I Need You
All I Need
Kingdom's Cause
'Way Down Yonder In New Orleans - Single Version
Kingdom Reigns
Sing It Out (He's Alive)
Crazy In Love (feat. Jay-Z)
Centuries
Seven Nation Army
Do I Wanna Know?
Here Comes The Sun - Remastered
Hey There Delilah
Sucker for Pain (with Wiz Khalifa, Imagine Dragons, Logic & Ty Dolla $ign feat. X Ambassadors)
X (feat. Future)
Closer
Cheerleader - Felix Jaehn Remix Radio Edit


/var/folders/v1/ljzyxbgj1xx_syzq56dgnsk00000gn/T/ipykernel_6883/3317257751.py:26: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  playlist_dict = rslt_df.set_index('playlist_name').T.to_dict('list')


...

/var/folders/v1/ljzyxbgj1xx_syzq56dgnsk00000gn/T/ipykernel_6883/3317257751.py:26: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  playlist_dict = rslt_df.set_index('playlist_name').T.to_dict('list')


I'm Yours is in ceremony
You Make Lovin' Fun is in ceremony
Dust in the Wind is in ceremony
The A Team is in ceremony
Two Hearts is in ceremony
Viva la Vida is in ceremony


HTTP Error for GET to https://api.spotify.com/v1/tracks/1b0cjXZlzGvKAf37sjii1s with Params: {'market': None} returned 404 due to Non existing id: 'spotify:track:1b0cjXZlzGvKAf37sjii1s'


You're the Inspiration is in ceremony
More Than Words is in ceremony
The Way You Look Tonight is in ceremony


SpotifyException: http status: 404, code:-1 - https://api.spotify.com/v1/tracks/1b0cjXZlzGvKAf37sjii1s:
 Non existing id: 'spotify:track:1b0cjXZlzGvKAf37sjii1s', reason: None

In [3]:
import fasttext 
from fasttext import train_supervised

# train fasttext model on the playlist names, using track uri as the label
model = fasttext.train_supervised(input='playlist.train', lr=0.025, epoch=5, wordNgrams=2, dim=100, loss='ova')


# save the model
model.save_model('improved_model.bin')

Read 2M words
Number of words:  7461
Number of labels: 483212
Progress: 100.0% words/sec/thread:     718 lr:  0.000000 avg.loss: 4125.361816 ETA:   0h 0m 0sss  5.8% words/sec/thread:    1086 lr:  0.023549 avg.loss: 5488.270020 ETA:   0h22m43s  8.0% words/sec/thread:     934 lr:  0.022995 avg.loss: 4897.180664 ETA:   0h25m48s  8.2% words/sec/thread:     932 lr:  0.022957 avg.loss: 4890.793945 ETA:   0h25m48s 12.6% words/sec/thread:     841 lr:  0.021850 avg.loss: 4383.362305 ETA:   0h27m13s 12.9% words/sec/thread:     837 lr:  0.021768 avg.loss: 4365.050781 ETA:   0h27m14s 13.8% words/sec/thread:     832 lr:  0.021554 avg.loss: 4345.012207 ETA:   0h27m 8s 20.8% words/sec/thread:     796 lr:  0.019794 avg.loss: 4026.451904 ETA:   0h26m 4s 23.4% words/sec/thread:     768 lr:  0.019146 avg.loss: 3966.201904 ETA:   0h26m 7sh26m 6s 23.8% words/sec/thread:     765 lr:  0.019052 avg.loss: 3958.226074 ETA:   0h26m 6s 23.9% words/sec/thread:     763 lr:  0.019033 avg.loss: 3956.414795 ETA:   0h2

In [15]:
import fasttext
# load the model
improved_model = fasttext.load_model('improved_model.bin')

# predict the track uri for a playlist name
track_predictions = improved_model.predict('feels', k=-1)

# get the track names for the predicted track uri's
for track in track_predictions[0]:
    print(spotify.track(track[9:])['name'])

Closer
The Hills
Roses (feat. ROZES)
Redbone
All The Small Things
Thinking out Loud
Mr. Brightside
Take Me To Church
Needed Me
Skinny Love
Despacito - Remix
Heathens
Slide (feat. Frank Ocean & Migos)
iSpy (feat. Lil Yachty)
Riptide
Electric Love
Love Yourself


KeyboardInterrupt: 

In [ ]:
# evaluate the model
improved_model.test("playlist.test", k=-1)

In [3]:
import fasttext 
# train fasttext model on the playlist names, using track uri as the label
model = fasttext.train_supervised('clean_playlist.txt')

# save the model
model.save_model('model.bin')

Read 2M words
Number of words:  8818
Number of labels: 558710
Progress: 100.0% words/sec/thread:     106 lr:  0.000000 avg.loss: 17.742762 ETA:   0h 0m 0s  9.6% words/sec/thread:    1183 lr:  0.090383 avg.loss: 17.751179 ETA:   0h25m17s 11.3% words/sec/thread:      16 lr:  0.088737 avg.loss: 17.751217 ETA:  29h21m17s     14 lr:  0.087394 avg.loss: 17.751291 ETA:  32h57m17s 13.2% words/sec/thread:      15 lr:  0.086812 avg.loss: 17.751320 ETA:  31h18m52s 20.3% words/sec/thread:      23 lr:  0.079706 avg.loss: 17.751553 ETA:  18h49m 4s 20.5% words/sec/thread:      23 lr:  0.079504 avg.loss: 17.751556 ETA:  18h35m24s 21.9% words/sec/thread:      25 lr:  0.078080 avg.loss: 17.751461 ETA:  17h 5m54s 25.6% words/sec/thread:      29 lr:  0.074411 avg.loss: 17.751249 ETA:  14h 0m15s 33.0% words/sec/thread:      37 lr:  0.067037 avg.loss: 17.750973 ETA:   9h51m23s 33.6% words/sec/thread:      38 lr:  0.066392 avg.loss: 17.750954 ETA:   9h34m47s 35.4% words/sec/thread:      40 lr:  0.064607 avg.

In [17]:
# load the model
model = fasttext.load_model('model.bin')

# predict the track uri for a playlist name
track_predictions = model.predict('rap happy', k=10)

# get the track names for the predicted track uri's
for track in track_predictions[0]:
    print(spotify.track(track[9:])['name'])
    


Closer
HUMBLE.
Stay With Me
Shape of You
See You Again (feat. Charlie Puth)
Where Are Ü Now (with Justin Bieber)
Counting Stars
This Is What You Came For
Starboy
Sorry


In [5]:
songs = ['15MTd64KUMG7CF6mOyovsQ', '3kGfazcbVvVkuZunzlLgTD', '7aF09WaavZAmAWuUeYxlYD', '1DfFHyrenAJbqsLcpRiOD9', '42gJat74CEyy3jPliCVKOC', '53SnCRcGrB2UCqNGyYZhJD', '1CtQClLffhYcQKRIAtM0qm', '4iLqG9SeJSnt0cSPICSjxv', '38yBBH2jacvDxrznF7h08J', '3B54sVLJ402zGa6Xm4YGNe', '6jA8HL9i4QGzsj6fjoxp8Y', '3PEgB3fkiojxms35ntsTgs', '5aAx2yezTd8zXrkmtKl66Z', '2QffgsYVZkvMQsZmQdgf2L', '2HSydyyFoQCBRxj3sZcbpz', '06TuMCjDEYjTOJScCRzmjf']

In [6]:
audio_features_dict = {}
for song in songs:
        audio_features_dict[song] = spotify.audio_features("spotify:track:"+ str(song))[0]

In [11]:
audio_features_df = pd.DataFrame.from_dict(audio_features_dict, orient='index')

audio_features_df = audio_features_df.drop(columns=['type', 'id','uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])
audio_features_df = audio_features_df.drop(columns=['key', 'mode', 'tempo'])

display(audio_features_df)

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence
15MTd64KUMG7CF6mOyovsQ,0.720,0.746,-5.567,0.0983,0.125000,0.000476,0.0929,0.511
3kGfazcbVvVkuZunzlLgTD,0.708,0.868,-3.287,0.0431,0.031300,0.000000,0.1220,0.720
7aF09WaavZAmAWuUeYxlYD,0.593,0.868,-3.573,0.2950,0.183000,0.000000,0.3650,0.496
1DfFHyrenAJbqsLcpRiOD9,0.597,0.567,-4.539,0.0342,0.561000,0.000005,0.1420,0.188
42gJat74CEyy3jPliCVKOC,0.768,0.858,-5.494,0.0482,0.191000,0.000032,0.6020,0.556
53SnCRcGrB2UCqNGyYZhJD,0.810,0.885,-4.583,0.1080,0.101000,0.000000,0.2680,0.447
1CtQClLffhYcQKRIAtM0qm,0.581,0.779,-6.178,0.0626,0.092500,0.000196,0.2300,0.172
4iLqG9SeJSnt0cSPICSjxv,0.774,0.626,-4.432,0.0432,0.096900,0.000031,0.0848,0.777
38yBBH2jacvDxrznF7h08J,0.734,0.418,-6.678,0.0425,0.012900,0.000000,0.0579,0.868
3B54sVLJ402zGa6Xm4YGNe,0.726,0.769,-5.043,0.1230,0.029300,0.010100,0.1040,0.733


In [47]:
# based on the audio features, create a similarity matrix
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(audio_features_df, audio_features_df)

# introduce some randomness so it skips some of the closest values, so that it doesn't pick the 
# absolute closest songs every time
# start with semnatic textual similarity 
# two vectors, compare how close they are to each other, could also just do an embedding 
# google colab gpu machines, or spin off an aws account, and spin off a gpu machine from aws or azure
# if you have access to the spotify api features, do it both ways
# once with word embeddings, once with music features, then combine them, depending on how much you want to weight them 

# create some metrics yourself for evaluation, or look at how many in playlists vs. outof plylist songs are recommended 
# vs. random sampling 

# signal processing for how to extract the classical features 

# 1: Get a sentence embedder
# 2: Put embeddings together in a latent space
# 3: Use KNN to get closest pairs
# 1: Get music features
# 2: Put music features in latent space
# 3: use KNN to get closest pairs
# 1: Combine two methods for song recommendations

# collaborative filtering

# create a series for the song titles so they are associated to an ordered numerical
# list I will use in the function to match the indexes
indices = pd.Series(audio_features_df.index)

# function that takes in song title as input and returns the top 10 recommended songs
def recommendations(song, cosine_sim = cosine_sim):
        
        recommended_songs = []
        
        # gettin the index of the song that matches the title
        idx = indices[indices == song].index[0]
    
        # creating a Series with the similarity scores in descending order
        score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    
        # getting the indexes of the 10 most similar songs
        top_10_indexes = list(score_series.iloc[1:11].index)
        
        # populating the list with the titles of the best 10 matching songs
        for i in top_10_indexes:
            recommended_songs.append(list(audio_features_df.index)[i])
            
        return recommended_songs

recommendations('15MTd64KUMG7CF6mOyovsQ')

# def wordMapfromSongVec(track_uri):
#     features = ['danceability','energy','speechiness','acousticness','liveness','valence']
#     songVec = spotify.audio_features("spotify:track:"+ str(track_uri))[0]
#     wordMap = {}
#     for feature in features:
#         wordMap[feature] = songVec[feature]
#     # turn audio features into text string where bigger values are more frequent
#     wordMap = {k: v * 100 for k, v in wordMap.items()}
    

['3PEgB3fkiojxms35ntsTgs',
 '2QffgsYVZkvMQsZmQdgf2L',
 '3B54sVLJ402zGa6Xm4YGNe',
 '5aAx2yezTd8zXrkmtKl66Z',
 '06TuMCjDEYjTOJScCRzmjf',
 '1CtQClLffhYcQKRIAtM0qm',
 '53SnCRcGrB2UCqNGyYZhJD',
 '38yBBH2jacvDxrznF7h08J',
 '4iLqG9SeJSnt0cSPICSjxv',
 '42gJat74CEyy3jPliCVKOC']